<a href="https://colab.research.google.com/github/shital-gupta7310/ai-chatbot-web-app/blob/main/CHATBOT_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask pyngrok mistralai

In [ ]:
  import os
  from flask import Flask
  from pyngrok import ngrok
  from mistralai import Mistral
  from getpass import getpass

  mistral_api_key = getpass("please enter your mistral api key")
  ngrok_auth_token = getpass("please enter your ngrok auth token")

please enter your mistral api key··········
please enter your ngrok auth token··········


In [ ]:
from flask import Flask, request, jsonify,render_template_string
bookmarks = []

history = []

client = Mistral(api_key=mistral_api_key)

app = Flask(__name__)
port = 8000


html = """
<!DOCTYPE html>
<html>
<head>
  <title>AI Chatbot</title>
  <style>
    * {
      box-sizing: border-box;
    }
    /* Theme Toggle Button */
#theme-toggle {
  position: absolute;
  top: 10px;
  right: 60px;
  padding: 6px 10px;
  font-size: 14px;
  border-radius: 5px;
  cursor: pointer;
  border: none;
  background: #ddd;
  transition: background 0.3s;
  z-index: 999;
}

#theme-toggle:hover {
  background: #bbb;
}

/* Dark mode styles */
body.dark {
  background: #1e1e1e;
  color: #f0f0f0;
}

body.dark #sidebar {
  background: #1b1b1b;
  border-right-color: #333;
}

body.dark #historyBox,
body.dark #bookmarkList {
  background: rgba(255, 255, 255, 0.05);
}

body.dark .chat-box {
  background: #2a2a2a;
  box-shadow: 0 4px 12px rgba(255, 255, 255, 0.05);
}

body.dark #chatbot {
  background: #1e1e1e;
}

body.dark .msg.bot {
  background: #333;
  color: #f0f0f0;
}

body.dark .msg.user {
  background: #007bff;
}

body.dark .input-area {
  background: #2d2d2d;
}

body.dark .input-area input {
  background: #444;
  color: #fff;
  border: 1px solid #555;
}

body.dark .input-area button {
  background: #28a745;
  color: white;
}

    body {
      margin: 0;
      padding: 0;
      font-family: Arial, sans-serif;
      background: #f0f2f5;
      display: flex;
      height: 100vh;
      overflow: hidden;

    }

    /* Sidebar */
    #sidebar {
      width: 250px;
      background: #2c3e50;
      color: white;
      padding: 20px;
      display: flex;
      flex-direction: column;
      border-right: 3px solid #1a252f;
    }

    #sidebar h3 {
      margin: 0 0 10px;
      padding-bottom: 5px;
      border-bottom: 1px solid rgba(255, 255, 255, 0.2);
      font-size: 18px;
    }

    #historyBox, #bookmarkList {
      flex: 1;
      overflow-y: auto;
      margin-bottom: 20px;
      padding: 10px;
      background: rgba(255, 255, 255, 0.1);
      border-radius: 5px;
      font-size: 14px;
    }

    #historyBox div, #bookmarkList div {
      padding: 5px;
      margin-bottom: 5px;
      background: rgba(0, 0, 0, 0.2);
      border-radius: 4px;
      word-wrap: break-word;
    }

    /* Chat Area */
    #chat-area {
      flex: 1;
      display: flex;
      justify-content: center;
      align-items: center;
      padding: 20px;
    }

    .chat-box {
      display: flex;
      flex-direction: column;
      background: white;
      width: 100%;
      max-width: 700px;
      height: 90%;
      border-radius: 10px;
      box-shadow: 0 4px 12px rgba(0, 0, 0, 0.15);
      overflow: hidden;
    }

    #chatbot {
      flex: 1;
      padding: 20px;
      overflow-y: auto;
      background: #fafafa;
    }

    .msg {
      margin: 10px 0;
      padding: 10px 15px;
      border-radius: 10px;
      max-width: 75%;
      line-height: 1.4;
      display: inline-block;
      clear: both;
    }

    .msg.user {
      background: #3498db;
      color: white;
      float: right;
    }

    .msg.bot {
      background: #ecf0f1;
      color: #333;
      float: left;
    }
   .msg {
  position: relative;  /* Important for absolute positioning of button */
  display: inline-block;
  margin: 10px 0;
  padding: 10px 15px;
  border-radius: 10px;
  max-width: 75%;
  line-height: 1.4;
}
.copy-btn {
  position: absolute;
  right: 8px;        /* Thoda spacing */
  top: 8px;          /* Top spacing */
  background: rgba(255, 255, 255, 0.8); /* Light background */
  border: 1px solid #ccc;
  border-radius: 4px;
  padding: 2px 5px;
  font-size: 13px;
  cursor: pointer;
  color: #333;
  transition: background 0.3s, color 0.3s;
}

.copy-btn:hover {
  background: #ddd;
  color: #000;
}


    /* Input Area */
    .input-area {
      display: flex;
      padding: 10px;
      background: #f7f7f7;
      border-top: 1px solid #ddd;
    }

    .input-area input {
      flex: 1;
      padding: 10px;
      border: 1px solid #ccc;
      border-radius: 5px;
      font-size: 14px;
    }

    .input-area button {
      margin-left: 10px;
      padding: 10px 15px;
      border: none;
      background: #2ecc71;
      color: light blue;
      border-radius: 5px;
      cursor: pointer;
      transition: background 0.3s;
    }

    .input-area button:hover {
      background: #27ae60;
    }

    /* Scrollbars */
    #chatbot::-webkit-scrollbar,
    #historyBox::-webkit-scrollbar,
    #bookmarkList::-webkit-scrollbar {
      width: 6px;
    }
    #chatbot::-webkit-scrollbar-thumb,
    #historyBox::-webkit-scrollbar-thumb,
    #bookmarkList::-webkit-scrollbar-thumb {
      background: rgba(0, 0, 0, 0.3);
      border-radius: 3px;
    }
  </style>
</head>
<body>
<button id="theme-toggle">🌙</button>

  <!-- Sidebar -->
  <div id="sidebar">
    <h3>📜 History</h3>
    <div id="historyBox"></div>
    <h3>📌 Bookmarks</h3>
    <div id="bookmarkList"></div>
  </div>

  <!-- Chat Area -->
  <div id="chat-area">
    <div class="chat-box">
      <div id="chatbot"></div>
      <div class="input-area">
        <input id="input" placeholder="Ask Anything...">
        <button onclick="send()">&rarr;</button>
      </div>
    </div>
  </div>


 <script>
  function send() {
    const msg = document.getElementById("input").value;
    if (!msg) return;

    add("You", msg, 'user');
    document.getElementById("input").value = '';

  fetch('/add_history', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({ sender: 'user', message: msg })
});


    fetch('./get_response', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ message: msg })
    })
    .then(res => res.json())
    .then(data => {
      add('Bot', data.response, 'bot');

     fetch('/add_history', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({ sender: 'bot', message: data.response })
});

      loadHistory();
    });
  }
 function add(sender, msg, cls) {
  const div = document.createElement('div');
  div.className = `msg ${cls}`;

  // Message text span
  const span = document.createElement('span');
  span.innerText = `${sender}: ${msg}`;
  div.appendChild(span);

  // Bookmark button for user messages
  if (cls === 'user') {
    const bmBtn = document.createElement('button');
    bmBtn.textContent = ' 🔖';
    bmBtn.style.marginLeft = '10px';
    bmBtn.style.cursor = 'pointer';
    bmBtn.onclick = function() {
      fetch('/bookmark', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ message: msg })
      }).then(() => loadBookmarks());
    };
    div.appendChild(bmBtn);
  }

  // Copy button for bot messages
  if (cls === 'bot') {
    const copyBtn = document.createElement('button');
    copyBtn.className = 'copy-btn';
    copyBtn.innerText = '📋';
    copyBtn.style.marginLeft = '10px';
    copyBtn.style.cursor = 'pointer';
    copyBtn.onclick = function() {
      navigator.clipboard.writeText(msg);
      copyBtn.innerText = '✔';
      setTimeout(() => copyBtn.innerText = '📋', 1500);
    };
    div.appendChild(copyBtn);
  }

  document.getElementById('chatbot').appendChild(div);
  document.getElementById('chatbot').scrollTop = document.getElementById('chatbot').scrollHeight;
}


  function loadHistory() {
  fetch('/history')
    .then(res => res.json())
    .then(questions => {
      const container = document.getElementById("historyBox");
      container.innerHTML = '';
      questions.forEach((q, index) => {
        const div = document.createElement('div');
        div.textContent = q;
        div.style.cursor = "pointer";
        div.onclick = () => loadConversation(q);
        container.appendChild(div);
      });
    });
}

function loadConversation(question) {
  fetch('/get_conversation', {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({ question })
  })
  .then(res => res.json())
  .then(data => {
    const botDiv = document.getElementById("chatbot");
    botDiv.innerHTML = ''; // Clear previous
    add("You", question, "user");
    add("Bot", data.answer, "bot");
  });
}


  function loadBookmarks() {
    fetch('/get_bookmarks')
      .then(res => res.json())
      .then(data => {
        const container = document.getElementById('bookmarkList');
        container.innerHTML = '';
        data.forEach((b, i) => {
  const d = document.createElement('div');
  d.style.display = "flex";
  d.style.justifyContent = "space-between";
  d.style.alignItems = "center";

  const textSpan = document.createElement('span');
  textSpan.textContent = `${i + 1}. ${b}`;

  const delBtn = document.createElement('button');
  delBtn.textContent = "❌";
  delBtn.style.cursor = "pointer";
  delBtn.onclick = function() {
    fetch('/delete_bookmark', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ index: i })
    }).then(() => loadBookmarks());
  };

  d.appendChild(textSpan);
  d.appendChild(delBtn);
  container.appendChild(d);
});


      });
  }

  function copyText(button) {
  const text = button.previousElementSibling.innerText;
  navigator.clipboard.writeText(text);
  button.innerText = "✔";
  setTimeout(() => button.innerText = "📋", 1500);
  }

  window.onload = function() {
    loadBookmarks();
    loadHistory();
  }
  // Theme Toggle Logic
const themeBtn = document.getElementById("theme-toggle");
const userPref = localStorage.getItem("theme");

if (userPref === "dark") {
  document.body.classList.add("dark");
  themeBtn.innerText = "☀️";
}

themeBtn.onclick = function () {
  document.body.classList.toggle("dark");
  const isDark = document.body.classList.contains("dark");
  themeBtn.innerText = isDark ? "☀️" : "🌙";
  localStorage.setItem("theme", isDark ? "dark" : "light");
};

</script>


</body>
</html>
"""


@app.route("/")
def index():
  return render_template_string(html)

@app.route('/get_response', methods=["POST"])
def get_response():
  user_input = request.json.get('message','')
  response = client.chat.complete(
      model="mistral-large-latest",
      messages = [{"role":'user',"content":user_input}]
  )
  return jsonify({'response': response.choices[0].message.content})

@app.route("/bookmark", methods=["POST"])
def bookmark():
    data = request.json
    msg = data.get('message', '')
    if msg:
        bookmarks.append(msg)
    return jsonify({'status': 'saved'})

@app.route("/get_bookmarks", methods=["GET"])
def get_bookmarks():
    return jsonify(bookmarks)

@app.route("/delete_bookmark", methods=["POST"])
def delete_bookmark():
  data = request.json
  index = data.get('index')
  if index is not None and 0 <= index < len(bookmarks):
    bookmarks.pop(index)
    return jsonify({'status': 'deleted'})
  return jsonify({'status': 'error', 'message': 'Invalid index'}), 400

@app.route('/add_history', methods=['POST'])
def add_history():
    msg = request.json.get('message')
    sender = request.json.get('sender')
    history.append({"sender": sender, "message": msg})
    return jsonify({"status": "added"})


@app.route('/history', methods=['GET'])
def get_history():
    # Return only user messages (questions)
    questions = [h["message"] for h in history if h["sender"] == "user"]
    return jsonify(questions)

@app.route('/get_conversation', methods=['POST'])
def get_conversation():
    question = request.json.get('question', '')
    answer = ''
    # Find the first bot message after the question
    for i, h in enumerate(history):
        if h["sender"] == "user" and h["message"] == question:
            # Look for the next bot reply
            if i + 1 < len(history) and history[i + 1]["sender"] == "bot":
                answer = history[i + 1]["message"]
            break
    return jsonify({"answer": answer})


if __name__ == "__main__":
  ngrok.set_auth_token(ngrok_auth_token)
  public_url = ngrok.connect(port)
  print("Chatbot running at: "+str(public_url))
  app.run(port=port)

Chatbot running at: NgrokTunnel: "https://ec960673087b.ngrok-free.app" -> "http://localhost:8000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:27] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:27] "GET /get_bookmarks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:27] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:27] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:34] "POST /add_history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:35] "POST /get_response HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:36] "POST /add_history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:36] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04:22:48] "POST /add_history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 04